In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
# Run this in a cell first
!pip install --upgrade tqdm

# Then restart kernel and run your training

In [5]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.1 MB/s eta 0:00:00


In [6]:
from datasets import DatasetDict, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification,TrainingArguments, Trainer
from datasets import load_dataset
import evaluate
from transformers import DataCollatorWithPadding
import torch

In [20]:
data_path = "/content/drive/MyDrive/WELFake_Dataset.csv"
DF = pd.read_csv(data_path)


In [21]:
DF.dropna()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1
5,5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,1
...,...,...,...,...
72129,72129,Russians steal research on Trump in hack of U....,WASHINGTON (Reuters) - Hackers believed to be ...,0
72130,72130,WATCH: Giuliani Demands That Democrats Apolog...,"You know, because in fantasyland Republicans n...",1
72131,72131,Migrants Refuse To Leave Train At Refugee Camp...,Migrants Refuse To Leave Train At Refugee Camp...,0
72132,72132,Trump tussle gives unpopular Mexican leader mu...,MEXICO CITY (Reuters) - Donald Trump’s combati...,0


In [22]:
DF.drop("Unnamed: 0" , inplace=True , axis=1)


In [23]:
DF["Content"] = DF["title"]+DF["text"]

In [24]:
DF.drop(["title" , "text"] , axis=1 , inplace=True)

In [25]:
DF.isna().sum()

,0
label,0
Content,597


In [26]:
DF.dropna(inplace=True)

In [27]:
DF.isna().sum()

,0
label,0
Content,0


In [28]:
import re

def clean(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    return text




In [29]:
DF

,label,Content
0,1,LAW ENFORCEMENT ON HIGH ALERT Following Threat...
2,1,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...
3,0,"Bobby Jindal, raised Hindu, uses story of Chri..."
4,1,SATAN 2: Russia unvelis an image of its terrif...
5,1,About Time! Christian Group Sues Amazon and SP...
...,...,...
72129,0,Russians steal research on Trump in hack of U....
72130,1,WATCH: Giuliani Demands That Democrats Apolog...
72131,0,Migrants Refuse To Leave Train At Refugee Camp...
72132,0,Trump tussle gives unpopular Mexican leader mu...


In [30]:
# Count duplicate rows based only on the "Content" column
num_duplicates = DF["Content"].duplicated().sum()
print(f"🔍 Number of duplicate content rows: {num_duplicates}")


🔍 Number of duplicate content rows: 8416


In [31]:

DF = DF.drop_duplicates(subset="Content").reset_index(drop=True)


In [32]:
DF

,label,Content
0,1,LAW ENFORCEMENT ON HIGH ALERT Following Threat...
1,1,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...
2,0,"Bobby Jindal, raised Hindu, uses story of Chri..."
3,1,SATAN 2: Russia unvelis an image of its terrif...
4,1,About Time! Christian Group Sues Amazon and SP...
...,...,...
63116,1,WIKILEAKS EMAIL SHOWS CLINTON FOUNDATION FUNDS...
63117,0,Russians steal research on Trump in hack of U....
63118,1,WATCH: Giuliani Demands That Democrats Apolog...
63119,0,Migrants Refuse To Leave Train At Refugee Camp...


In [33]:
DF["Content"] = DF["Content"].apply(clean)

In [36]:
# Count duplicate rows based only on the "Content" column
num_duplicates = DF["Content"].duplicated().sum()
print(f"🔍 Number of duplicate content rows: {num_duplicates}")


🔍 Number of duplicate content rows: 0


In [37]:

DF = DF.drop_duplicates(subset="Content").reset_index(drop=True)


In [38]:
DF = DF[DF["Content"].str.strip() != ""]


In [39]:
DF

,label,Content
0,1,law enforcement on high alert following threat...
1,1,unbelievable obamas attorney general says most...
2,0,bobby jindal raised hindu uses story of christ...
3,1,satan 2 russia unvelis an image of its terrify...
4,1,about time christian group sues amazon and spl...
...,...,...
63077,1,wikileaks email shows clinton foundation funds...
63078,0,russians steal research on trump in hack of us...
63079,1,watch giuliani demands that democrats apologiz...
63080,0,migrants refuse to leave train at refugee camp...


In [40]:
DF_dataset = Dataset.from_pandas(DF)

In [41]:
accuracy = evaluate.load("accuracy")
auc_score = evaluate.load("roc_auc")

def compute_metrics(eval_pred):
    # get predictions
    predictions, labels = eval_pred

    # apply softmax to get probabilities
    probabilities = np.exp(predictions) / np.exp(predictions).sum(-1,
                                                                 keepdims=True)
    # use probabilities of the positive class for ROC AUC
    positive_class_probs = probabilities[:, 1]
    # compute auc
    auc = np.round(auc_score.compute(prediction_scores=positive_class_probs,
                                     references=labels)['roc_auc'],3)

    # predict most probable class
    predicted_classes = np.argmax(predictions, axis=1)
    # compute accuracy
    acc = np.round(accuracy.compute(predictions=predicted_classes,
                                     references=labels)['accuracy'],3)

    return {"accuracy": acc, "auc": auc}

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [42]:
model_path = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_path)

# load model with binary classification head
id2label = {0: "Fake", 1: "Real"}
label2id = {"Fake": 0, "Real": 1}
model = AutoModelForSequenceClassification.from_pretrained(model_path,
                                                           num_labels=2,
                                                           id2label=id2label,
                                                           label2id=label2id,)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [43]:
# Freeze all
for param in model.bert.parameters():
    param.requires_grad = False

# Unfreeze last 2 encoder layers
for param in model.bert.encoder.layer[-2:].parameters():
    param.requires_grad = True


In [44]:
print(DF_dataset)
print(DF_dataset.column_names)


Dataset({
    features: ['label', 'Content', '__index_level_0__'],
    num_rows: 63081
})
['label', 'Content', '__index_level_0__']


In [ ]:
def preprocess_function(examples):
    texts = examples["Content"]
    # Ensure all elements are strings
    texts = [str(t) if t is not None else "" for t in texts]
    return tokenizer(texts, truncation=True, padding=True, max_length=512)

# preprocess all datasets
tokenized_data = DF_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/63081 [00:00<?, ? examples/s]

In [19]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [20]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/my_bert_model",  # ✅ Save checkpoints to Drive
    eval_strategy="epoch",                        # ✅ Corrected key name
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    warmup_ratio=0.1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",              # ✅ Matches lowercase metric
    greater_is_better=True,
    logging_steps=50,
    save_total_limit=2,
    disable_tqdm=False,
    report_to=None,
    fp16=True  # ✅ (Optional: mixed-precision for faster training on GPU)
)


In [21]:
import transformers
print(f"Transformers version: {transformers.__version__}")

Transformers version: 4.52.4


In [22]:
# 80% train, 20% test
train_test = tokenized_data.train_test_split(test_size=0.2, seed=42)


In [23]:
# Split the 20% test into half validation, half test => 10% val, 10% test
val_test = train_test['test'].train_test_split(test_size=0.5, seed=42)


In [24]:
# Combine all splits into a DatasetDict
final_dataset = DatasetDict({
    'train': train_test['train'],
    'validation': val_test['train'],
    'test': val_test['test']
})


In [99]:
len(final_dataset["validation"])

7213

In [97]:
# Convert to sets of strings for easy comparison
train_texts = set(final_dataset["train"]["Content"])
val_texts = set(final_dataset["validation"]["Content"])

# Find common entries
overlap = train_texts.intersection(val_texts)
print(f"🔍 Overlapping samples: {len(overlap)}")


🔍 Overlapping samples: 1299


In [25]:
from tqdm.auto import tqdm
print("✅ tqdm is ready!")


✅ tqdm is ready!


In [26]:
# Quick test to see if progress bars work
from tqdm.auto import tqdm
import time

print("Testing progress bar...")
for i in tqdm(range(10), desc="Test"):
    time.sleep(0.1)
print("✅ Progress bar working!")

Testing progress bar...


Test:   0%|          | 0/10 [00:00<?, ?it/s]

✅ Progress bar working!


In [27]:
from transformers import EarlyStoppingCallback
# Create final dataset

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=final_dataset["train"],
    eval_dataset=final_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],  # ✅ Stops if no improvement for 2 evals
)


# Start training - should now show progress bars!
print("🚀 Starting training with progress bars...")
trainer.train(resume_from_checkpoint=True)

/tmp/ipython-input-27-701422774.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


🚀 Starting training with progress bars...


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: vaibhavbarala8 (vaibhavbarala8-iit) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss


TrainOutput(global_step=36070, training_loss=0.0, metrics={'train_runtime': 43.2989, 'train_samples_per_second': 6663.794, 'train_steps_per_second': 833.046, 'total_flos': 7.59167483582976e+16, 'train_loss': 0.0, 'epoch': 5.0})

In [28]:
# ✅ Final model save path in Drive
final_save_path = f"{training_args.output_dir}/final_model"

print("Saving model...")
trainer.save_model(final_save_path)
tokenizer.save_pretrained(final_save_path)

print("Training completed!")
print(f"Best model and tokenizer saved in: {final_save_path}")

# Optional: Evaluate on test set


Saving model...
Training completed!
Best model and tokenizer saved in: /content/drive/MyDrive/my_bert_model/final_model


In [29]:
#trainer.train(resume_from_checkpoint=True)

In [71]:
final_dataset["test"]

Dataset({
    features: ['label', 'Content', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 7214
})

In [72]:
# Evaluate on validation set (or test set if you have one)
eval_results = trainer.evaluate(eval_dataset=final_dataset["test"])

print("📊 Evaluation Results:")
for key, value in eval_results.items():
    print(f"{key}: {value:.4f}")


📊 Evaluation Results:
eval_loss: 0.0204
eval_accuracy: 0.9960
eval_auc: 1.0000
eval_runtime: 59.4367
eval_samples_per_second: 121.3730
eval_steps_per_second: 15.1760
epoch: 5.0000


In [40]:
DF_test = pd.read_csv("./news_dataset.csv")

In [41]:
DF_test.isna().sum()

,0
label,0
text,8


In [42]:
DF_test.dropna()

,label,text
0,REAL,Payal has accused filmmaker Anurag Kashyap of ...
1,FAKE,A four-minute-long video of a woman criticisin...
2,FAKE,"Republic Poll, a fake Twitter account imitatin..."
3,REAL,"Delhi teen finds place on UN green list, turns..."
4,REAL,Delhi: A high-level meeting underway at reside...
...,...,...
3724,REAL,19:17 (IST) Sep 20\n\nThe second round of coun...
3725,REAL,19:17 (IST) Sep 20\n\nThe second round of coun...
3726,FAKE,The Bengaluru City Police’s official Twitter h...
3727,REAL,"Sep 20, 2020, 08:00AM IST\n\nSource: TOI.in\n\..."


In [43]:
DF_test["label"] = np.where(DF_test["label"] == "REAL" , 1 , 0)

In [75]:
DF_test["Content"] = DF_test["text"]

In [76]:
DF_test["Content"].apply(clean)

,Content
0,payal has accused filmmaker anurag kashyap of ...
1,a fourminutelong video of a woman criticising ...
2,republic poll a fake twitter account imitating...
3,delhi teen finds place on un green list turns ...
4,delhi a highlevel meeting underway at residenc...
...,...
3724,1917 ist sep 20 the second round of countrywid...
3725,1917 ist sep 20 the second round of countrywid...
3726,the bengaluru city polices official twitter ha...
3727,sep 20 2020 0800am ist source toiin meet neelk...


In [77]:
import re

def clean_test(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)  # keep numbers too
    text = re.sub(r'\s+', ' ', text).strip()

    return text


In [79]:
DF_test["Content"] = DF_test["Content"].apply(clean_test)

In [80]:
DF_test.drop(["text"] , axis=1 , inplace=True)

In [81]:
DF_test

,label,Content
0,1,payal has accused filmmaker anurag kashyap of ...
1,0,a fourminutelong video of a woman criticising ...
2,0,republic poll a fake twitter account imitating...
3,1,delhi teen finds place on un green list turns ...
4,1,delhi a highlevel meeting underway at residenc...
...,...,...
3724,1,1917 ist sep 20 the second round of countrywid...
3725,1,1917 ist sep 20 the second round of countrywid...
3726,0,the bengaluru city polices official twitter ha...
3727,1,sep 20 2020 0800am ist source toiin meet neelk...


In [82]:
DF_dataset_test = Dataset.from_pandas(DF_test)

In [83]:
tokenized_data_test = DF_dataset_test.map(preprocess_function, batched=True)

Map:   0%|          | 0/3729 [00:00<?, ? examples/s]

In [84]:
result = trainer.predict(tokenized_data_test)

PredictionOutput(predictions=array([[-5.484375 ,  6.1210938],
       [-4.0546875,  4.4765625],
       [-5.6523438,  6.2851562],
       ...,
       [-5.765625 ,  6.40625  ],
       [-5.6992188,  6.3398438],
       [-5.671875 ,  6.328125 ]], dtype=float32), label_ids=array([1, 0, 0, ..., 0, 1, 1]), metrics={'test_loss': 5.885051250457764, 'test_accuracy': 0.482, 'test_auc': 0.317, 'test_runtime': 31.6724, 'test_samples_per_second': 117.737, 'test_steps_per_second': 14.745})

In [86]:
result = trainer.predict(tokenized_data_test)

In [87]:
pred = np.argmax(result.predictions, axis=-1)

In [91]:
np.mean(pred == DF_test["label"])

np.float64(0.48216680075087154)